# Merging and Cleaning Data

In this section, the obtained data over the past few days will be merged together. We start by opening all CSV's with gaming and review data that were collected over the past few days.

In [ ]:
base_directory = Path(__file__).parent if "__file__" in globals() else Path().resolve()

num_days = 7
start_date = datetime.strptime('20250302', '%Y%m%d')

date_list = [(start_date + timedelta(days=i)).strftime('%Y%m%d') for i in range(num_days)]

games_data = {}
reviews_data = {}

for date in date_list:
    search_result_folder_path = base_directory / f"search_results_{date}"  
    games_path = search_result_folder_path / f"globaltopsellers_{date}.csv"
    review_path = search_result_folder_path / f"globaltopsellers_reviews_{date}.csv"

    if games_path.exists():
        games_data[date] = pd.read_csv(games_path)
        games_data[date]["extraction_date"] = datetime.strptime(date, "%Y%m%d")
        print(f"Loaded games data for {date}")

    if review_path.exists():
        reviews_data[date] = pd.read_csv(review_path)
        reviews_data[date]["extraction_date"] = datetime.strptime(date, "%Y%m%d")
        print(f"Loaded reviews data for {date}")

## Cleaning and Changing the Game Data

First, we change the formatting of the price that is displayed on our dataset to remove the symbols. 

In [ ]:
def convert_price(price):
    try:
        clean_price = ''.join([ch for ch in str(price) if ch.isdigit() or ch == '.'])
        return float(clean_price) if clean_price else None
    except Exception:
        return None

If a game is free, `price_overview` will show an NA value. Therefore, we have decided to set the prices of games that are free to the value 0. We have also calculated the `discount_ratio` of games that are currently on sale, again, changing the values of games that are free -- therefore, returning NA value -- as 0. 

In [ ]:
for day in date_list:
    if day in games_data:  
        games = games_data[day] 

        ## CONVERT PRICE
        games['original_price_numeric'] = games['original_price'].apply(convert_price)
        games['discounted_price_numeric'] = games['discounted_price'].apply(convert_price)
        games['price_overview_numeric'] = games['price_overview'].apply(convert_price)

        ## REPLACE MISSING VALUES WITH 0 FOR FREE GAMES
        games.loc[games['is_free'] == True, ['original_price_numeric', 'discounted_price_numeric', 'price_overview_numeric']] = 0

        ## CALCULATE DISCOUNT RATIO
        games['discount_ratio'] = np.where(
            games['original_price_numeric'] > 0,
            (games['original_price_numeric'] - games['discounted_price_numeric']) / games['original_price_numeric'],
            0  # If original price is zero, set discount ratio to 0
        )

        ## REPLACE NA VALUES WITH 0
        games['discount_ratio'] = games['discount_ratio'].fillna(0)

        ## LOG-TRANSFORM PRICE -- HANDLE PRICE VARIABILITY ##
        games['log_price'] = np.log1p(games['original_price_numeric'])  # log(price + 1) to avoid log(0)
        
        games_data[day] = games

    else:
        print(f"games_{day} does not exist!")

## Cleaning and Changing Review Data

The variable `voted_up` returns `TRUE` or `FALSE` value. Next, we convert the `voted_up` value to binary `1` for "Recommended" and `0` for "Not Recommended".

Then, we convert `playtime_at_review` and `num_games_owned` to numeric. These two variables represent the hours spent playing the game before leaving a review and the number of games the user owns, respectively.

Lastly, we calculate the `weighted_positive` which is the variable we will be using forward to measure whether a user positively recommends the game or not. This variable is a result of calculating the variable `voted_up` with the `weighted_vote_score` -- which is essentially the rate of influence the review has with `1` as the maximum value. 

For example:
- `app_id`: 2246340
- `recommendationid`: 189066635
- `steamid`: 76561198106660720
- `voted_up`: `TRUE`
- `weighted_vote_score`: 0.872241854667663574

$$ \text{weighted\_positive} = \text{voted\_up} \times \text{weighted\_vote\_score} $$
$$ \text{weighted\_positive} = 1 \times 0.872241854667663574 = 0.872241854667663574 $$

In [ ]:
for day in date_list:
    if day in reviews_data:  
        review = reviews_data[day]
    
        ## CONVERT VOTED_UP TO BOOLEAN
        review['voted_up_bool'] = review['voted_up'].astype(bool)
        review['voted_up_binary'] = review['voted_up'].astype(int)  # Convert to binary (1 = Recommended, 0 = Not Recommended)

        ## CONVERT TO NUMERIC
        review['playtime_at_review'] = pd.to_numeric(review['playtime_at_review'], errors='coerce')
        review['num_games_owned'] = pd.to_numeric(review['num_games_owned'], errors='coerce')

        ## WEIGHTED POSITIVITY SCORE 
        review['weighted_positive'] = review['voted_up'] * review['weighted_vote_score']
        
        reviews_data[day] = review

    else:
        print(f"review_{day} does not exist!")

## Merging Data

In [ ]:
for day in date_list:
    if day in reviews_data and day in games_data:
        review = reviews_data[day]
        games = games_data[day] 

        ## MERGE ON APPID
        merged_review = review.merge(games, on='appid', how='inner')

        reviews_data[day] = merged_review

    else:
        print(f"Either review or games data for {day} does not exist!")

Now that we have a full data frame for each day that we collected data. We merge all the data removing duplicates. We do it in this order, since some columns of both Games and Review can change over time. We want to connect the values that were captured on a certain day, which is why we first join the data frames for a specific day and then merge all dates together. We remove full duplicates, meaning if every single column has the same value as another row, one of the rows is deleted. 

In [ ]:
# COLLECT ALL REVIEWS DATA PER DATE
all_reviews = []

for day in date_list:
    if day in reviews_data:  
        all_reviews.append(reviews_data[day])
    else:
        print(f"review data for {day} not found!")

## COMBINE AND REMOVE FULL DUPLICATES 
if all_reviews:
    combined_reviews = pd.concat(all_reviews, ignore_index=True).drop_duplicates()
else:
    combined_reviews = pd.DataFrame() 
    
combined_reviews.head()

Then we remove the duplicates by considering these three columns only. That means any rows where recommendationid, steamid, and appid are the same (regardless of differences in other columns) will be dropped, leaving only the first occurrence.

In [ ]:
combined_reviews.drop_duplicates(subset=["recommendationid", "steamid", "appid"], keep = "first", inplace = True)

Then, we remove several columns that are not needed anymore.

In [ ]:
columns_removed = ['logo', 'timestamp_created', 'timestamp_updated', 'type', 'steam_appid', 'supported_languages', 'developers', 'publishers', 'platforms', 'num_reviews']
combined_reviews = combined_reviews.drop(columns = columns_removed)

Then we save the final dataset into a CSV file.

In [ ]:
combined_reviews.to_csv("combined_reviews.csv", index = False)